# 多模态大模型之 CLIP与Llava

In [1]:
from collections import OrderedDict
from typing import Tuple, Union

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## 0 开篇：多模态大模型与CLIP

多模态大模型（Multimodal Large Model）是一种能够处理和理解多种模态数据的人工智能模型。模态指的是数据的表现形式，例如文字、图像、音频、视频等。多模态大模型通过结合不同模态数据的特性，利用深度学习技术和大规模的训练数据，构建一个统一的框架来实现跨模态的感知、理解和生成能力。例如，一个多模态大模型可以同时处理文本描述和对应的图片，完成图像生成、描述生成、跨模态检索等任务。这种模型的核心是利用不同模态之间的互补性，通过信息的融合和关联增强模型的表现能力，克服单一模态信息不足的问题。

学习多模态大模型的意义在于其广泛的应用潜力和巨大的社会价值。首先，多模态大模型可以在真实世界中广泛应用于多场景任务，如自动驾驶中的视觉和语言指令结合、医疗影像分析中的图像和文本报告结合、智能客服中的语音与文字结合等。这种模型能更好地模拟人类在复杂情境中处理多种信息的能力，从而大幅提升人工智能系统的实用性和智能水平。其次，多模态大模型的研究促进了人工智能跨学科的融合，推动了计算机视觉、自然语言处理、语音识别等领域的协同发展。此外，多模态学习也对人机交互的自然性和效率有重要提升作用，帮助构建更直观、更人性化的交互系统。总之，学习和发展多模态大模型是实现通用人工智能的重要一步，能够推动技术的前沿进展，为各行各业带来深远的影响。

关于多模态模型，我们许多疑惑之处，在正式学习之前，我们要先辨析部分关于多模态大模型的概念。

- **Q1：多模态为什么有价值？从工程的角度来说，为什么多模态模型难度高？**

多模态模型能够处理更丰富的信息表达、现实又大量跨模态任务需求，且多模态很有可能是同往AGI的道路，这是多模态模型最有价值的点之一。然而，多模态数据的最大挑战在于 **它们不在一个空间中**，每种模态的结构和分布差异巨大，难以直接结合：

- **图像数据**：
    - 结构：4维 (N, C, H, W)。
      - N：批量大小（batch size）。
      - C：通道数（通常为 3，RGB）。
      - H 和 W：图像高度和宽度。
    - 特性：数据通常是局部相关（如相邻像素相关），需要卷积操作提取局部特征。
<br><br>
- **时间序列数据**：
    - 结构：3维 (N, T, F)。
      - N：批量大小。
      - T：时间步长。
      - F：特征维度（如传感器数据的特征数）。
    - 特性：时间相关性强，依赖递归网络（如 RNN）或 Transformer 提取时间序列特征。
<br><br>
- **文本数据**：
    - 结构：2维 (N, L) 或 3维 (N, L, D)
      - N：批量大小。
      - L：句子长度（不同样本可能长度不同）。
    - 特性：离散数据，依赖序列建模（如 Transformer）理解上下文。
<br><br>
- **视频数据**：
    - 结构：5维 (N, T, C, H, W)。
      - N：批量大小。
      - T：时间帧数。
      - C：通道数。
      - H 和 W：帧的高度和宽度。
    - 特性：视频数据包含时间和空间信息，既需要卷积提取空间特征，又需要时序建模处理时间相关性。
<br><br>
- **音频数据**：
    - 结构：3维 (N, T, F)。
      - N：批量大小。
      - T：时间步长。
      - F：频谱特征（如 Mel 频谱的频率维度）。
    - 特性：通常是时间相关数据，需要结合卷积和序列建模。

- **Q2：都是多模态大模型、SORA/Veo2和LLava/Flamingo有什么不同？**

多模态模型是能够处理多种模态（如文本、图像、视频等）数据的人工智能模型，但在细分领域中，我们通常把多模态模型分为两大类：**多模态理解模型**和**多模态生成模型**。 

1. **多模态理解模型**：这类模型的输入是多模态数据（如图像和文本），但输出不一定是多模态的，通常是一个预测结果或单一模态的数据（如分类标签或文字）。
   - **例子**：
     - **CLIP**：输入图像和文本，用于在共享语义空间中匹配图像和文本，输出相似度或分类结果。
     - **LLaVA**：输入图像和问题文本，输出自然语言回答。
     - **Flamingo**：能够将视觉和文本信息结合，用于多模态对话或问答。
     - **ActionCLIP**：视频理解模型，输入视频和文本描述，输出动作分类结果。
<br><br>
2. **多模态生成模型**：这类模型不仅可以理解多模态信息，其输出也可以是多模态的（如图像生成、视频生成或音频生成）。  
   - **例子**：
     - **DALL·E**：输入文本描述，生成对应的图像。
     - **Stable Diffusion**：输入文本描述，生成高质量图像。
     - **SORA**：输入视频和文本，生成符合语义的视频片段。
     - **VideoGPT**：输入语义信息，生成动态的视频内容。
     - **GATO**：支持输入和输出都是多模态的任务，例如从图像输入到动作输出。

这些模型在多模态处理上展现了强大的能力，多模态理解模型通常专注于**模态间语义的对齐和推理**，多模态生成模型**则进一步实现了从语义到多模态内容的生成**。两类模型在任务和应用中互为补充，推动了多模态 AI 的快速发展。<font color ="Red">**特别的是，从结构上看，多模态生成模型可以理解为多模态理解模型 + 一个生成解码器**</font>，因为生成模型不仅需要理解输入，还需要在解码阶段根据输入模态生成输出模态。下面是两种多模态架构的对比——

| 特点                  | 多模态理解模型                                                                                                                                                  | 多模态生成模型                                                                                                                                               |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| **输入**           | 通常是多模态数据，例如图像、文本、视频等形式的数据，提供给模型进行理解任务。                                                                                     | 可以是多模态数据，也可以是单一模态数据，作为生成过程的条件输入，例如文本或图像。                                                                                       |
| **输出**           | 输出单一模态的预测结果，例如分类标签、相似度分数、或自然语言形式的回答等。                                                                                          | 输出为生成的多模态数据，包括图像、视频、文本等内容，具有更广泛的表现形式。                                                                                           |
| **架构核心**       | 1. **模态编码器（Encoders）**：每种模态独立的编码器处理输入数据，例如 CNN 或 ViT（图像），Transformer 或预训练语言模型（文本）。编码器将不同模态的数据转化为高维特征向量。<br><br>2. **模态对齐模块（Alignment Module）**：通过对比学习或跨模态注意力机制，将不同模态的特征对齐到共享的语义空间。例如，CLIP 利用对比学习对齐图像和文本的语义表示。<br><br>3. **输出头（Prediction Head）**：根据具体任务输出结果，例如分类标签、相似度分数或文本回答等内容。<br><br>- **例子**：<br>- **CLIP**：图像编码器 + 文本编码器 + 对比学习目标。<br>- **Flamingo**：视觉编码器结合文本编码器，通过注意力机制实现模态对齐。 | 1. **模态编码器（Encoders）**：和多模态理解模型类似，生成模型也有独立的模态编码器，例如文本编码器提取语义嵌入，作为生成条件输入。<br><br>2. **生成解码器（Generative Decoder）**：生成模型增加了解码器，将编码的特征转化为目标模态。<br><br>- *图像生成*：解码器通过扩散模型（如 Stable Diffusion）或 GAN 生成图像。<br>- *视频生成*：通过时间序列扩展（如 VideoGPT）生成动态视频内容。<br>- *文本生成*：使用标准 Transformer 解码器生成文本描述。<br><br>3. **跨模态交互模块（Cross-Modal Interaction Module）**：生成过程中，可能需要模态之间反复交互，例如跨模态注意力机制。例如，DALL·E 使用文本特征指导生成图像，确保生成结果符合输入语义。<br><br>- **例子**：<br>- **DALL·E**：文本编码器结合扩散解码器生成图像。<br>- **Stable Diffusion**：文本编码器结合图像生成解码器生成精细图像。<br>- **VideoGPT**：多模态编码器与视频解码器结合生成动态视频。 |


总结一下，我们可以认为理解模型和生成模型有的差异就在于——

- **理解模型**：输入数据 → 编码器 → 共享语义空间 → 输出单一模态预测。
- **生成模型**：输入数据 → 编码器 → 跨模态交互模块 → 解码器 → 输出多模态内容。

- **Q3：多模态模型与多模态大模型有什么区别？多大才算是多模态大模型？**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/05.png)

补充：DALLE2约有3.5B参数左右、而llava则分为8B、13B、34B三种尺寸。

- **Q4：多模态大模型的训练和推理需要多少运存？什么样的设备才能跑？**

训练和推理多模态大模型（如参数规模为 0.4B 的 CLIP 模型）对硬件资源有一定要求。以下是对所需内存和设备的详细说明：

**训练阶段：**

- **显存需求：** 训练 CLIP 模型时，显存需求取决于模型大小、批次大小（batch size）和优化器状态等因素。以 0.4B 参数的 CLIP 模型为例，使用较小的批次大小（如 32）进行训练，可能需要约 16GB 或更多的显存。增大批次大小或使用更复杂的优化器将增加显存需求。

- **设备要求：** 高端 GPU，如 NVIDIA RTX 3090（24GB 显存）或更强的 GPU（如 A100），适合训练此类模型。此外，分布式训练可以利用多块 GPU 或多台机器来分担计算和内存负担。

**推理阶段：**

- **显存需求：** 推理时，显存需求相对较低，但仍需考虑模型大小和输入数据的尺寸。对于 0.4B 参数的 CLIP 模型，8GB 至 12GB 显存的 GPU 通常足够进行推理。

- **设备要求：** 中高端 GPU，如 NVIDIA GTX 1080 Ti、RTX 2080 Ti 或更高型号，能够满足推理需求。在某些情况下，使用 CPU 也可以进行推理，但速度会明显慢于 GPU。

**注意事项：**

- **内存需求：** 除了显存，主机的 RAM 也需要足够大，以处理数据加载和预处理等任务。建议至少具备 16GB RAM，较大的内存（如 32GB 或更多）有助于提升性能。

- **存储空间：** 模型文件和数据集可能占用数十 GB 的存储空间。确保有足够的磁盘空间来存储模型权重和相关数据。

- **软件环境：** 确保安装了兼容的深度学习框架（如 PyTorch）和必要的库。根据 OpenAI 的 CLIP 项目，所需的 Python 库包括 `torch`、`torchvision`、`ftfy`、`regex`、`tqdm` 和 `packaging`。 

请注意，实际的硬件需求可能因具体的模型配置、数据集大小和应用场景而有所不同。在部署前，建议根据自身的任务需求进行测试和评估，以确定所需的硬件资源。 

- **Q5：为什么是CLIP？为什么从CLIP开始？**

CLIP（Contrastive Language-Image Pre-training）是OpenAI于2021年发布的一种**多模态预训练模型**，它能够将图像和文本嵌入到同一向量空间中，实现同一模型系统下的跨模态理解。其基本结构包括图像编码器和文本编码器，分别处理图像和文本数据。图像编码器通常采用卷积神经网络（如ResNet）或Vision Transformer（ViT），而文本编码器则使用Transformer模型。这些编码器分别处理图像和文本数据，并将它们投影到一个共同的嵌入空间中、从而实现图文分类、文本-图像检索等基础的多模态任务。

对于CLIP，我们今天要学习和掌握它的关键原因在于——

1. **独创性的架构**：CLIP是一个无监督的、端到端的多模态系统，它是图文同向量空间对齐思路下效果最好的模型之一，它采用了经典的Transformer架构，是我们理解深度学习世界星罗棋布的多模态算法的根基。<br><br>
2. **独创性的预训练流程**：CLIP创新了精彩的预训练方式对比学习预训练（contrastive learning）、这种训练方式不仅不需要对数据进行标注、还使得多模态模型的训练样本从“特定任务数据”拓展到了“”使得CLIP能够在下游任务中实现零样本学习，即在未见过的类别上进行分类或检索、使其在文本生成图像描述、跨模态搜索、图文匹配等任务中展现了前所未有的灵活性和效率、大大拓展了多模态模型的应用场景。<br><br>
3. **Llava必备基础**：多模态大模型LLava借鉴了CLIP的思想与板块，因此理解CLIP可以帮我们更好地学习Llava模型<br><br>
4. **非常好的顶会文章基础**：光是2024年一年内，牛津大学、复旦大学、上海佳通大学、以及微软研究院、Google研究院都分别以CLIP为核心主题发表了CVPR论文，CLIP可以说是非常好的链接深度学习与大语言模型的关键研究点。

-----

截至2024年，关于CLIP（Contrastive Language-Image Pre-training）的研究在多个顶级会议上发表。以下是一些具有代表性的论文及其链接：

1. **Alpha-CLIP: A CLIP Model Focusing on Wherever You Want**  
   该论文提出了Alpha-CLIP模型，旨在增强CLIP在特定图像区域的关注能力，使其能够更精确地处理用户指定的图像区域。  
   *发表会议：2024年IEEE计算机视觉与模式识别会议（CVPR 2024）*  
   [论文链接](https://openaccess.thecvf.com/content/CVPR2024/html/Sun_Alpha-CLIP_A_CLIP_Model_Focusing_on_Wherever_You_Want_CVPR_2024_paper.html)

2. **MoDE: CLIP Data Experts via Clustering**  
   该研究提出了MoDE方法，通过对数据进行聚类，训练一组CLIP数据专家，以提升模型在多样化数据上的表现。  
   *发表会议：2024年IEEE计算机视觉与模式识别会议（CVPR 2024）*
   [论文链接](https://openaccess.thecvf.com/content/CVPR2024/html/Ma_MoDE_CLIP_Data_Experts_via_Clustering_CVPR_2024_paper.html)

3. **CLIP-ProbCR: CLIP-based Probability Embedding Combination Retrieval**
    该论文提出了一个使用文本和图像组合进行检索的图像检索模型，利用 CLIP 编码文本与图像，获得对应的特征，通过对特征添加概率分布获得特征的高斯概率密度函数，最后使用概率密度函数之间的乘法法则获得组合特征的表示。
    发表会议：2024 年多媒体检索国际会议（ICMR 2024）
    [论文链接](https://dl.acm.org/doi/10.1145/3652583.3657611)

4. **CLIP as RNN: Segment Countless Visual Concepts without Training Endeavor**
    该论文基于CLIP提出了全新的零样本图像分割方法，通过循环框架将 CLIP 的开放词汇能力与分割任务相结合，逐步过滤无关文本并提高掩码质量、这样既拓展了 CLIP 的应用范围，也突破了现有图像分割方法的局限性。
    *发表会议：2024年IEEE计算机视觉与模式识别会议（CVPR 2024）*
    [论文链接](https://arxiv.org/abs/2312.07661)

5. **Highlight-CLIP: Unleash the Potential of CLIP for Video Highlight Detection**  
   该论文提出了Highlight-CLIP方法，利用CLIP的预训练知识，提升视频高光检测任务的性能。  
   *发表会议：2024年IEEE计算机视觉与模式识别会议工作坊（CVPRW 2024）*  
   [论文链接](https://openaccess.thecvf.com/content/CVPR2024W/ELVM/html/Han_Unleash_the_Potential_of_CLIP_for_Video_Highlight_Detection_CVPRW_2024_paper.html)

6. **MobileCLIP: Fast Image-Text Models through Multi-Modal Reinforced Training**
    论文提出了一个名为 MobileCLIP 的高效图文模型家族。MobileCLIP 针对 CLIP 模型的特性和性能瓶颈（如计算资源需求和延迟问题），进行了优化，以便能够更好地适应移动设备等资源受限的环境，同时，论文设计了一种多模态强化训练（multi-modal reinforced training）方法，充分利用了 CLIP 模型的能力，这些工作属于对 CLIP 模型的优化和工程改进，并充分体现了 CLIP 模型在多模态领域的强大潜力。
   *发表会议：2024年IEEE计算机视觉与模式识别会议（CVPR 2024）*  
    [论文链接](https://arxiv.org/abs/2311.17049)

还有的大量我没有去记录期刊的论文——

1. **FLEX-CLIP: Feature-Level Generation Network Enhanced CLIP for X-shot Learning**  
   该论文介绍了FLEX-CLIP模型，通过特征级生成网络增强CLIP的能力，旨在解决小样本学习中的数据不平衡和特征退化问题。  
   *发表会议/期刊：信息待补充*  
   [论文链接](https://arxiv.org/abs/2411.17454)

2. **TripletCLIP: Improving Compositional Reasoning of CLIP via Triplet Loss**  
   该研究提出了TripletCLIP方法，通过引入三元组损失，增强CLIP的组合推理能力，在复杂概念理解上取得了显著提升。  
   *发表会议/期刊：信息待补充*  
   [论文链接](https://arxiv.org/abs/2411.02545)

3. **FastCLIP: A Suite of Optimization Techniques to Accelerate CLIP Training with Limited Resources**  
   该论文提出了FastCLIP框架，旨在通过一系列优化技术，在有限资源下加速CLIP的训练过程。  
   *发表会议/期刊：信息待补充*  
   [论文链接](https://arxiv.org/abs/2407.01445)

4. **Scaling (Down) CLIP: A Comprehensive Analysis of Data, Architecture, and Training Strategies**  
   该研究对CLIP的缩放进行了全面分析，探讨了数据、架构和训练策略对模型性能的影响，为资源受限情况下的模型训练提供了指导。  
   *发表会议/期刊：信息待补充*  
   [论文链接](https://arxiv.org/abs/2404.08197)

5. **CLIP in Medical Imaging: A Comprehensive Survey**  
    该综述论文系统地总结了CLIP在医学影像领域的应用，展示了其在多种临床任务中的潜力和挑战。  
    *发表会议/期刊：信息待补充*  
    [论文链接](https://arxiv.org/abs/2312.07353)

总而言之CLIP是具备跨模态检索、零样本分类能力的跨时代突破性架构，是多模态学习领域的根基，今天就让我们一起来看看这个经典的多模态模型。

## 1 CLIP模型架构

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/01.png)

从这张图可以看出，CLIP的模型架构包含三个主要部分：**图像编码器**、**文本编码器** 和 **余弦相似度计算模块**。CLIP 是一个强大的多模态对比学习模型，通过图像编码器和文本编码器分别对图像和文本进行特征提取，并在共享语义空间中对齐两种模态，最后通过余弦相似度实现跨模态任务的统一处理。

### 1.1 图像编码器

在 **CLIP**（Contrastive Language-Image Pretraining）中，选择使用 **残差网络（ResNet）** 或 **视觉Transformer（ViT）** 取决于任务需求、计算资源、模型复杂度以及性能目标。两者各有优劣——

ResNet作为CLIP的图像编码器，适用于以下情况：

- **轻量化场景**：  
  ResNet的计算效率较高，尤其在参数量和计算复杂度受限时，其优化的卷积设计和残差连接使其在嵌入计算效率和性能之间取得较好平衡。

- **小规模数据和有限资源**：  
  如果计算资源有限或需要快速部署，ResNet的高效架构更适合，因为它训练速度较快，并且对小数据集的泛化能力较强。

- **对局部特征依赖较强的任务**：  
  ResNet通过卷积操作擅长提取图像的局部特征，因此在局部信息对模型性能至关重要的任务中表现较好。

- **经典的卷积架构迁移**：  
  ResNet是传统卷积神经网络的代表，许多现成的优化方法和技术（如权重初始化、批归一化、迁移学习）都能直接套用，便于快速调整和扩展。

---

ViT作为CLIP的图像编码器，适用于以下情况：

- **大规模数据集（如LAION-400M等）**：  
  ViT对大规模训练数据表现更佳，因为其自注意力机制能够捕获更丰富的全局信息，并且在数据充足的情况下展现出极强的特征学习能力。

- **对全局特征建模需求高的任务**：  
  ViT通过Transformer架构的自注意力机制，可以更好地捕获图像的全局关系，因此在需要语义对齐或全局信息理解的任务中效果更强。

- **追求更高性能和模型可扩展性**：  
  ViT在模型扩展性和性能上有优势，尤其是在高计算资源场景中，ViT能够利用更深层次的注意力机制提升图像表征能力。

- **跨领域迁移能力强**：  
  ViT具备较好的跨领域迁移能力，在图像特征需要进一步跨模态对齐或迁移到其他任务时表现出色。

---

- **数据规模和计算资源**：  
  如果数据规模较小，或者计算资源受限，优先选择ResNet。如果有足够的数据和计算资源，则ViT表现更优。

- **特征建模需求**：  
  ResNet更适合依赖局部特征的任务，而ViT更适合全局语义建模。

- **部署需求**：  
  ResNet因其较低的计算需求和较高的部署成熟度，适合需要快速、轻量部署的场景；而ViT适合在追求性能极限的离线或高性能计算环境中使用。

在CLIP中，OpenAI已经实现了**基于ResNet和ViT的两种模型版本**，用户可以根据任务需求选择最适合的编码器架构。

#### 1.1.1 残差网络

- **stem**

在深度学习中，Stem 是指网络的起始部分（通常是前几层），用于处理原始输入数据（如图像）并将其转换为更适合后续处理的特征表示。Stem通常由一些简单的卷积操作或其他特征提取模块组成，旨在快速压缩输入数据的维度，并提取初步特征。

残差网络（ResNet）中的Stem
在 ResNet 中，Stem模块是模型的最前端部分，通常负责对原始图像进行基本处理。这部分的设计决定了输入图像的特征提取效率以及后续网络的性能和计算复杂度。

- **基础的残差网络结构与残差层**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/02.png)

In [2]:
class Bottleneck(nn.Module):
    # 定义扩展倍数，用于决定输出通道数的扩展比例
    expansion = 4

    def __init__(self, inplanes, planes, stride=1):
        """
        Bottleneck 模块的初始化方法。
        参数：
        - inplanes: 输入通道数。
        - planes: 每一层卷积的基础通道数，最终会扩展为 planes * expansion。
        - stride: 步幅，控制空间分辨率的缩小（默认值为1）。
        """
        super().__init__()

        # 第1个卷积层，使用 1x1 卷积，用于减少计算量
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 批归一化
        self.relu1 = nn.ReLU(inplace=True)  # 激活函数，inplace=True 节省内存

        # 第2个卷积层，使用 3x3 卷积，用于提取特征
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 批归一化
        self.relu2 = nn.ReLU(inplace=True)  # 激活函数

        # 如果 stride > 1，使用 AvgPool2d 进行空间降采样，否则使用 Identity（保持不变）
        self.avgpool = nn.AvgPool2d(stride) if stride > 1 else nn.Identity()

        # 第3个卷积层，使用 1x1 卷积，用于扩展通道数至 planes * expansion
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)  # 批归一化
        self.relu3 = nn.ReLU(inplace=True)  # 激活函数

        # 如果输入和输出的通道数或分辨率不同，则需要通过 downsample 进行调整
        self.downsample = None
        self.stride = stride

        if stride > 1 or inplanes != planes * Bottleneck.expansion:
            # downsample 模块，用于调整 shortcut 的形状，使其与主路径的输出形状匹配
            self.downsample = nn.Sequential(OrderedDict([
                # 使用 AvgPool2d 进行空间降采样（仅在 stride > 1 时）
                ("-1", nn.AvgPool2d(stride)),
                # 1x1 卷积调整通道数
                ("0", nn.Conv2d(inplanes, planes * self.expansion, kernel_size=1, stride=1, bias=False)),
                # 批归一化
                ("1", nn.BatchNorm2d(planes * self.expansion))
            ]))

    def forward(self, x: torch.Tensor):
        """
        前向传播函数。
        参数：
        - x: 输入张量，形状为 (batch_size, inplanes, H, W)
        返回值：
        - 输出张量，形状为 (batch_size, planes * expansion, H / stride, W / stride)
        """
        identity = x  # 保存输入作为 shortcut（残差连接的输入）

        # 主路径：第1个卷积 + BN + ReLU
        out = self.relu1(self.bn1(self.conv1(x)))
        # 主路径：第2个卷积 + BN + ReLU
        out = self.relu2(self.bn2(self.conv2(out)))
        # 如果 stride > 1，进行空间降采样
        out = self.avgpool(out)
        # 主路径：第3个卷积 + BN
        out = self.bn3(self.conv3(out))

        # 如果 downsample 不为 None，则对 identity 进行调整
        if self.downsample is not None:
            identity = self.downsample(x)

        # 残差连接：将主路径的输出和 shortcut 相加
        out += identity
        # 激活函数
        out = self.relu3(out)
        return out

- **AttentionPool 注意力池化**

Attention Pooling 是 CLIP 的 **Modified ResNet** 架构中用来替代传统全局平均池化（Global Average Pooling, GAP）的方法、同时对整个CLIP来说也是重要的**向量压缩层**。在经过了 `stem` 和多个 `Residual Layers`（`layer1-layer4`）的特征提取后，网络会输出一个形状为 `(batch_size, channels, height, width)` 的特征图，Attention Pooling 会将这个特征图整合为一个全局特征向量，表示整幅图像的特征、从而方便后续的投影。因此Attention Pooling 的主要任务是将二维的特征图 (batch_size, channels, height, width) 转换为一个全局特征向量 (batch_size, embed_dim)。

和传统的全局平均池化相比，注意力池化可以**对特征图上的每个位置（像素点）的重要性**进行加权、这样生成的全局特征向量能够更好地捕获空间上的信息以及全局上下文关系。我们来看一下Attention Pooling 的具体实现代码（同时纯享版代码在py文件中哦）——

In [3]:
class AttentionPool2d(nn.Module):
    def __init__(self, spacial_dim: int, embed_dim: int, num_heads: int, output_dim: int = None):
        """
        初始化 Attention Pooling 模块。

        参数：
        - spacial_dim (int): 输入特征图的空间维度（即特征图宽高的大小，假设特征图是正方形）。
        - embed_dim (int): 输入特征的嵌入维度（通道数）。
        - num_heads (int): 多头注意力的头数。
        - output_dim (int, optional): 输出特征的目标维度。如果未指定，则默认为 embed_dim。
        """
        super().__init__()
        # 位置嵌入 (Positional Embedding)，初始化为随机值，大小为 (spacial_dim^2 + 1, embed_dim)
        # 额外的 "+1" 是为全局 token 预留的位置。
        self.positional_embedding = nn.Parameter(torch.randn(spacial_dim ** 2 + 1, embed_dim) / embed_dim ** 0.5)

        # Query、Key、Value 的线性投影
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)

        # 输出投影，将嵌入维度从 embed_dim 调整为 output_dim
        self.c_proj = nn.Linear(embed_dim, output_dim or embed_dim)

        # 多头注意力的头数
        self.num_heads = num_heads

    def forward(self, x):
        """
        前向传播函数，执行 Attention Pooling 操作。

        参数：
        - x (Tensor): 输入特征图，形状为 (batch_size, channels, height, width)。

        返回：
        - Tensor: 输出的全局特征向量，形状为 (batch_size, output_dim)。
        """
        # 1. 展平特征图并转置
        # 原始形状：x -> (batch_size, channels, height, width)
        # 转换后：x -> (height * width, batch_size, channels)
        x = x.flatten(start_dim=2).permute(2, 0, 1)

        # 2. 添加全局 token
        # 全局 token 是整幅特征图的均值，用于捕获全局信息。
        # `x.mean(dim=0, keepdim=True)` 的形状为 (1, batch_size, channels)
        # 添加到特征图的前面，新的 x 形状为 (height * width + 1, batch_size, channels)
        x = torch.cat([x.mean(dim=0, keepdim=True), x], dim=0)

        # 3. 添加位置嵌入
        # self.positional_embedding 的形状为 (height * width + 1, embed_dim)
        # 将位置嵌入添加到特征图上，每个位置都会加上对应的嵌入值。
        x = x + self.positional_embedding[:, None, :].to(x.dtype)

        # 4. 执行多头注意力
        # 注意力操作的参数：
        # - query=x[:1]: 使用全局 token 作为 Query，形状为 (1, batch_size, embed_dim)。
        # - key=x: 使用整个特征图（包括全局 token）作为 Key，形状为 (height * width + 1, batch_size, embed_dim)。
        # - value=x: 同上，使用整个特征图作为 Value。
        # 注意力机制的输出形状为 (1, batch_size, embed_dim)，即仅保留全局 token 的注意力输出。
        x, _ = F.multi_head_attention_forward(
            query=x[:1],  # 全局 token 作为 Query
            key=x,  # 整个特征图作为 Key
            value=x,  # 整个特征图作为 Value
            embed_dim_to_check=x.shape[-1],  # 嵌入维度
            num_heads=self.num_heads,  # 注意力头的数量
            q_proj_weight=self.q_proj.weight,  # Query 的投影权重
            k_proj_weight=self.k_proj.weight,  # Key 的投影权重
            v_proj_weight=self.v_proj.weight,  # Value 的投影权重
            in_proj_weight=None,  # 不使用合并的投影权重
            in_proj_bias=torch.cat([self.q_proj.bias, self.k_proj.bias, self.v_proj.bias]),  # Query, Key, Value 的偏置
            bias_k=None,  # Key 的额外偏置（未使用）
            bias_v=None,  # Value 的额外偏置（未使用）
            add_zero_attn=False,  # 不添加额外的全零注意力头
            dropout_p=0,  # 不使用 Dropout
            out_proj_weight=self.c_proj.weight,  # 输出投影的权重
            out_proj_bias=self.c_proj.bias,  # 输出投影的偏置
            use_separate_proj_weight=True,  # 使用单独的 Query, Key, Value 投影权重
            training=self.training,  # 根据当前模式确定是否训练
            need_weights=False  # 不需要返回注意力权重
        )

        # 5. 移除第一个维度
        # 注意力输出的形状为 (1, batch_size, embed_dim)，去掉第一个维度后，变为 (batch_size, embed_dim)。
        return x.squeeze(0)

- **输入：**
  - 特征图 `(batch_size, channels, height, width)`。
  - 特征图被展平到 `(spatial_dim, batch_size, embed_dim)` 以适配注意力机制。
<br><br>
- **处理步骤：**<br><br>
  1. **Flatten 特征图**:
     - 将特征图展平（flatten），从 `(N, C, H, W)` 变为 `(HW, N, C)`，即空间位置作为序列。
  2. **添加全局 token**:
     - 为每个图像添加一个全局 token，代表全局的上下文信息。
  3. **位置嵌入 (Positional Embedding)**:
     - 为每个位置加上位置信息，保留空间结构。
  4. **多头注意力机制**:
     - 使用多头注意力，计算每个位置的重要性权重，并整合为全局特征向量。
  5. **输出变换**:
     - 通过线性投影层调整最终输出的维度（`output_dim`）。
<br><br>
- **输出：**
  - 一个全局特征向量，表示整幅图像的特征。

In [4]:
# 假设输入数据
batch_size = 4  # 假设有 4 个样本
channels = 512  # 从 Bottleneck 输出的特征通道
height = 7      # 特征图的高度
width = 7       # 特征图的宽度
spacial_dim = height  # 假设输入特征图是正方形
embed_dim = channels  # 嵌入维度与输入通道数一致
num_heads = 8  # 多头注意力头数
output_dim = 256  # 输出特征维度

# 生成模拟输入数据（模拟从 Bottleneck 输出的特征图）
input_data = torch.randn(batch_size, channels, height, width)

In [5]:
input_data.shape

torch.Size([4, 512, 7, 7])

In [6]:
# 创建 AttentionPool2d 实例
attention_pool = AttentionPool2d(spacial_dim, embed_dim, num_heads, output_dim)

# 前向传播
output = attention_pool(input_data)

In [7]:
output.shape, output

(torch.Size([4, 256]),
 tensor([[ 0.0342,  0.0482,  0.0279,  ..., -0.0458,  0.0388,  0.1614],
         [-0.0745,  0.0592,  0.0461,  ..., -0.0438,  0.0673,  0.0063],
         [ 0.0225,  0.0971,  0.0892,  ..., -0.0572,  0.0491,  0.0798],
         [-0.0118,  0.1063,  0.0430,  ...,  0.0047, -0.0379, -0.0116]],
        grad_fn=<SqueezeBackward1>))

- **完整的残差网络实现**

In [8]:
class ModifiedResNet(nn.Module):
    """
    一个基于 ResNet 的改进版本，包含以下变化：
    - 起始部分使用了 3 层卷积（stem），而不是原始 ResNet 的单层卷积，同时将 max pool 替换为 avg pool。
    - 对于 stride > 1 的卷积层，使用了反锯齿（anti-aliasing）的卷积方式，即在卷积前添加了 avg pool。
    - 最终的全局池化层替换为 QKV 注意力池化层，而不是传统的 average pool。
    """

    def __init__(self, layers, output_dim, heads, input_resolution=224, width=64):
        """
        初始化函数。
        参数：
        - layers: 每个阶段的残差块数量（如 [3, 4, 6, 3] 表示每个阶段的残差块数）。
        - output_dim: 输出特征的维度。
        - heads: 注意力池化的头数。
        - input_resolution: 输入图像的分辨率（默认 224）。
        - width: 初始通道宽度（默认 64）。
        """
        super().__init__()
        self.output_dim = output_dim
        self.input_resolution = input_resolution

        # 初始化阶段（stem）：3 层卷积代替原来的单层卷积
        self.conv1 = nn.Conv2d(3, width // 2, kernel_size=3, stride=2, padding=1, bias=False)  # 输入通道为3，输出为 width//2
        self.bn1 = nn.BatchNorm2d(width // 2)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(width // 2, width // 2, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(width // 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(width // 2, width, kernel_size=3, padding=1, bias=False)  # 输出为 width
        self.bn3 = nn.BatchNorm2d(width)
        self.relu3 = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(2)  # 平均池化，用于下采样

        # 残差层
        self._inplanes = width  # 初始输入通道数，用于动态调整残差块的输入
        self.layer1 = self._make_layer(width, layers[0])  # 第一阶段残差块
        self.layer2 = self._make_layer(width * 2, layers[1], stride=2)  # 第二阶段残差块，输出通道翻倍
        self.layer3 = self._make_layer(width * 4, layers[2], stride=2)  # 第三阶段残差块
        self.layer4 = self._make_layer(width * 8, layers[3], stride=2)  # 第四阶段残差块

        # 注意力池化层
        embed_dim = width * 32  # ResNet 的特征维度，等于最后一个阶段的输出通道数
        self.attnpool = AttentionPool2d(input_resolution // 32, embed_dim, heads, output_dim)  # QKV 注意力池化

    def _make_layer(self, planes, blocks, stride=1):
        """
        创建一个残差模块（由多个 Bottleneck 组成）。
        参数：
        - planes: 当前阶段的通道数。
        - blocks: 残差块数量。
        - stride: 当前阶段的步幅。
        返回值：
        - 一个由多个残差块组成的序列模块。
        """
        layers = [Bottleneck(self._inplanes, planes, stride)]  # 第一个残差块，可能需要调整步幅
        self._inplanes = planes * Bottleneck.expansion  # 更新通道数
        for _ in range(1, blocks):
            layers.append(Bottleneck(self._inplanes, planes))  # 后续残差块保持步幅为 1

        return nn.Sequential(*layers)  # 返回残差块的序列

    def forward(self, x):
        """
        前向传播函数。
        参数：
        - x: 输入张量，形状为 (batch_size, channels, height, width)。
        返回值：
        - x: 输出张量，经过特征提取和注意力池化后。
        """
        def stem(x):
            """
            起始阶段（stem）：连续 3 层卷积 + 平均池化。
            参数：
            - x: 输入张量。
            返回值：
            - x: 经过 stem 阶段后的特征张量。
            """
            x = self.relu1(self.bn1(self.conv1(x)))  # 第1层卷积
            x = self.relu2(self.bn2(self.conv2(x)))  # 第2层卷积
            x = self.relu3(self.bn3(self.conv3(x)))  # 第3层卷积
            x = self.avgpool(x)  # 平均池化
            return x

        x = x.type(self.conv1.weight.dtype)  # 将输入类型调整为卷积权重的类型（支持混合精度训练）
        x = stem(x)  # 经过起始阶段
        x = self.layer1(x)  # 第一阶段残差层
        x = self.layer2(x)  # 第二阶段残差层
        x = self.layer3(x)  # 第三阶段残差层
        x = self.layer4(x)  # 第四阶段残差层
        x = self.attnpool(x)  # 最后通过注意力池化
        return x

In [10]:
# 实例化模型
model = ModifiedResNet(layers=[3, 4, 6, 3], output_dim=512, heads=8)

# 输入示例数据
input_data = torch.randn(10, 3, 224, 224)  # (batch_size, channels, height, width)
output = model(input_data)

print("输出形状:", output.shape)

输出形状: torch.Size([10, 512])


如你所见，残差网络已经将数据结构转变为了一个二维向量。

#### 1.1.2 Vision Transformer

**Vision Transformer (ViT)** 是一种将 Transformer 架构应用于计算机视觉任务的深度学习模型，旨在利用 Transformer 的全局建模能力处理图像数据。与传统的卷积神经网络（CNN）不同，ViT 不依赖卷积操作，而是通过将图像分割为固定大小的子块（Patch）并将这些子块视为序列输入，结合自注意力机制对全局特征进行建模。ViT 的主要特点在于，它能够直接处理图像的全局上下文关系，尤其在大规模数据集上（如 ImageNet 或 LAION）展现了极高的性能。

ViT的特性就是能够处理图像信息、并将图像信息转变为Transformer能够读取的信息。在理解Transformer和卷积层的基础上，ViT并不难，让我们来解答关于ViT的几个最为核心的问题——

- <font color="red">**Q1：ViT如何将图像信息转变为Transformer能够读取的序列？**

在 ViT 中，负责转变信息的结构叫做**Patch Embedding**。传统图像输入是三维的张量（如形状为 $ H \times W \times C $，分别表示图像高度、宽度和通道数），而 Transformer 的输入要求是一维的序列。因此，**Patch Embedding 的任务是将图像转化为适合 Transformer 输入的嵌入序列**。具体来说，ViT 会将输入图像按照固定大小 $ P \times P $ 分割成若干个不重叠的子块（Patch），每个 Patch 被展平为一维向量，形状为 $ P^2 \cdot C $，其中 $ C $ 是图像的通道数。随后，使用一个线性投影层（通常是一个全连接层）将每个 Patch 的一维向量投影到 $ D $ 维的特征空间中（$ D $ 是 Transformer 的隐藏层维度），生成对应的 Patch Embedding。我们可以来看一下具体的流程——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/06_1.png)

如你所见，ViT是将每个分开的patch中所有的像素整合成了一个向量，通过这种方式就可以让图像信息被转变为Transformer能够接纳的三维向量。这个过程用代码实现是 ↓ 

In [42]:
import torch
import torch.nn as nn

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        """
        初始化 PatchEmbedding 类，用于将输入图像划分为若干 Patch，并投影到固定的嵌入维度。
        
        参数：
        - img_size: 输入图像的分辨率（默认为 224）。
        - patch_size: 每个 Patch 的尺寸（默认为 16x16）。
        - in_channels: 输入图像的通道数（默认为 3，即 RGB 图像）。
        - embed_dim: 嵌入维度（Transformer 的输入特征维度，默认为 768）。
        """
        super().__init__()
        self.img_size = img_size  # 保存输入图像的尺寸
        self.patch_size = patch_size  # 保存每个 Patch 的尺寸
        self.num_patches = (img_size // patch_size) * (img_size // patch_size)  
        # 计算图像被划分的总 Patch 数量 = (图像宽度上的 Patch 数) * (图像高度上的 Patch 数)
        self.patch_dim = patch_size * patch_size * in_channels  
        # 每个 Patch 的维度 = patch_size * patch_size * 通道数（即展平后的特征数）

        # 定义一个线性层，将每个 Patch 的原始特征（patch_dim）投影到 Transformer 的嵌入维度（embed_dim）
        self.projection = nn.Linear(self.patch_dim, embed_dim)
        # 初始化一个可学习的参数，用于位置编码，形状为 (1, num_patches, embed_dim)
        self.position_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))

    def forward(self, x):
        """
        前向传播函数，将输入图像转换为 Patch 嵌入，并添加位置编码。

        参数：
        - x: 输入张量，形状为 (batch_size, in_channels, height, width)。

        返回值：
        - embeddings: 包含位置编码的 Patch 嵌入，形状为 (batch_size, num_patches, embed_dim)。
        """
        # 获取输入张量的批量大小、通道数、高度和宽度
        B, C, H, W = x.shape
        # 确保输入图像的分辨率与定义的 img_size 一致
        assert H == self.img_size and W == self.img_size, "Input size must match img_size"
        
        # 将输入图像划分为 Patch，使用 unfold 函数：
        # unfold(2, patch_size, patch_size) 在高度维度上以 patch_size 的窗口滑动并提取小块；
        # unfold(3, patch_size, patch_size) 在宽度维度上以 patch_size 的窗口滑动并提取小块。
        patches = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        
        # contiguous() 确保数据在内存中是连续的；
        # view 将张量重塑为 (B, num_patches, patch_dim)，
        # 其中每个 Patch 被展平为长度为 patch_dim 的向量。
        patches = patches.contiguous().view(B, -1, self.patch_dim)

        # 使用线性层将每个 Patch 的特征投影到嵌入维度（embed_dim），
        # 输出的形状为 (B, num_patches, embed_dim)。
        embeddings = self.projection(patches)

        # 添加位置编码，位置编码的形状为 (1, num_patches, embed_dim)，
        # 会广播到 batch 的每个样本。
        embeddings += self.position_embedding

        # 返回添加了位置编码的 Patch 嵌入
        return embeddings

然而在CLIP中，存在更高效的做法——**即利用卷积层来完成上述流程**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/06_2.png)

这个过程由代码实现起来非常简单，只需要将out_channels设置为d_model，并且将卷积核kernel_size和步长stride设置为patch_size即可 ↓ 

In [27]:
d_model = 512
patch_size = 16
stride = 16

conv1 = nn.Conv2d(in_channels=3, out_channels=d_model, kernel_size=patch_size, stride=patch_size, bias=False)

In [37]:
input_data = torch.randn(10, 3, 224, 224)  # (batch_size, channels, height, width)

In [38]:
x = conv1(input_data)

In [41]:
x.shape #最终输出的是 224/16 = 14

torch.Size([10, 512, 14, 14])

In [35]:
x = x.reshape(x.shape[0], x.shape[1], -1)  # 形状 = [batch_size, width, grid ** 2]
x = x.permute(0, 2, 1)  # 调整为 [batch_size, num_patches, d_model]

In [36]:
x.shape

torch.Size([10, 196, 512])

总结一下 ↓ 

| Patch Embedding的环节                 | ViT常规实现方法 | CLIP中的实现方法|
|-----------------------|-----|------|
| **分块**           | 设置好patch_size<br>使用unfold等方式手动划分 | 使用卷积层进行划分 |
| **投影前展平**           | 在划分好的patch上继续unford<br>view等方式进行手动展平 | -- |
| **投影**           | 设置单独的投影线性层<br>接收in_channels * patch_size * patch_size<br>输出d_model | 在卷积层中<br>通过将out_channels设置成d_model<br>从而实现投影 |
| **投影后展平**       | -- | 使用 reshape和permute进行展平|


这么做的优势在于——

**1. 更高的计算效率**
使用卷积操作可以有效利用硬件加速：
- **并行计算**：卷积操作在现代深度学习框架（如 PyTorch、TensorFlow）中得到了高度优化，可以充分利用 GPU 的并行计算能力，而展平每个 Patch 的传统方式需要额外的显式循环操作，效率较低。
- **少数据搬移**：传统方式需要将图像数据重新组织成 $ [num\_patches, patch\_size \cdot patch\_size \cdot channels] $，这涉及到大量的数据搬移。而卷积操作直接在输入图像上执行，无需显式展平，减少了数据搬移的开销。

---

**2. 自然的空间感知**
卷积操作天生具有局部感知能力：
- 卷积核会在每个 Patch 内进行加权求和，相当于提取 Patch 的局部特征，而不是简单的展平数据。
- 如果需要，卷积还可以在 Patch 中实现更高级的特征提取（比如使用非线性激活函数或更深的卷积层），相比于简单的 `Linear` 投影更加灵活。

---

**3. 内存消耗更少**
卷积操作直接作用于原始的高维图像输入，而传统方法中展平操作会生成一个高维度的中间结果：
- 在传统实现中，每个 $16 \times 16$ 的 Patch 会被展平成长度为 $16 \times 16 \times 3 = 768$ 的向量。如果图像分辨率较高，Patch 较多，会导致内存占用增加。
- 使用 `Conv2d` 的实现直接输出 $ [batch\_size, width, grid\_height, grid\_width] $，避免了显式展平产生的中间结果，从而节省内存。

---

**4. 代码简洁性与可扩展性**
卷积的实现直接将 Patch 的划分和投影合并成一个操作：
- **简洁性**：`Conv2d` 的 `kernel_size` 和 `stride` 参数天然对应 Patch 的大小和步幅，代码更加直观、简洁，省去了展平和逐个 Patch 投影的步骤。
- **可扩展性**：如果需要更复杂的处理，可以直接在卷积层中添加更多的非线性操作（如激活函数）或深层网络结构，而不是依赖额外的模块。

---

**5. 支持任意大小的输入图像**
- **传统方法**：要求图像分辨率是 Patch 大小的整数倍（否则划分出的 Patch 数量不一致），而且需要显式处理 Padding。
- **卷积方法**：`Conv2d` 中的 `padding` 参数能够自动处理边界情况，使得网络支持任意大小的输入图像。

---

**6. 与预训练 CNN 特征结合的潜力**
在实际应用中，我们可能会用预训练的 CNN 特征作为 Transformer 的输入：
- 卷积操作本质上是 CNN 的核心，因此可以方便地与预训练模型（如 ResNet、EfficientNet）的特征对接。
- 这种方式可以充分利用卷积的特性提取更高质量的 Patch 特征。

---

**7. 灵活的 Patch 特征生成**
- **传统方法**：展平的 Patch 中，每个像素值直接作为输入，无法动态调整 Patch 内部的特征。
- **卷积方法**：通过设计卷积核（如不同大小、深度或非线性处理），可以灵活生成更复杂的 Patch 特征。例如：
  - 使用更大的卷积核捕获跨越多个像素的上下文关系；
  - 使用多层卷积构造更丰富的层次化 Patch 表示。

---

**8. 提前降维，减少计算量**
如果输入图像分辨率较高（如 4K 图像），直接展平后的序列长度会非常长，导致 Transformer 的计算量急剧增加。而卷积方法可以在切片的同时降低通道数，从而有效减少计算量。

- <font color="red">**Q2：除了Patch Embedding之外，ViT和普通的Transformer还有什么不同？**

为了保留图像中 Patch 的位置信息，ViT 会在 Patch Embedding 中加入**二维位置编码（Positional Encoding）**，以向模型提供顺序信息。这种位置编码可以是可学习的参数，或者是基于正弦和余弦的固定位置编码。最终，所有 Patch Embedding 和位置编码的加和结果被组成一个序列，输入 Transformer 模型进行后续的全局特征建模和分类任务。通过这种方式，ViT 将原始图像处理成了一个序列结构，充分利用了 Transformer 在自然语言处理任务中擅长的序列建模能力，从而在图像分类、目标检测等视觉任务中实现了优异的性能。

除此之外，**ViT支持类别嵌入**。类别嵌入是一个 learnable 参数，直接添加到序列的开头，用于提取全局图像特征，就类似于我们在Bert当中所插入的[CLS]、这个token/patch用于收集全局信息。

同时，ViT所需要的**数据量会比普通Transformer大很多**、ViT被认为是一个图像的架构而不是文本的架构，因此ViT需要的输入是Patch、而普通Transformer需要的输入是Token。

- <font color="red">**Q3：常规的ViT和CLIP中的ViT有什么区别？**

| 特性                       | 普通 ViT                | CLIP中的ViT            |
|----------------------------|-------------------------|------------------------------|
| LayerNorm                  | 标准 LayerNorm          | 在前向传播中，先将输入数据转换为 `float32` 类型<br>计算 LayerNorm，再恢复为原始数据类型（如 `fp16`）<br>支持一定程度的混合精度训练。<br><br>通过这种方式，避免了使用 `fp16` 时因数值精度不足<br>导致的归一化误差，提升训练稳定性。 |
| 前馈网络                   | 标准 GELU 前馈网络              | 使用了自定义的 **QuickGELU** 激活函数，其公式为：<br><br>$\text{QuickGELU}(x) = x \cdot \text{sigmoid}(1.702 \cdot x)$<br><br>QuickGELU比标准 GELU 计算效率更高<br>且在实际性能上接近或优于标准 GELU。                   |
| 类别嵌入初始化             | 无特殊处理             | 使用 `scale = width ** -0.5` 对初始值进行缩放<br>缩放初始化，增强稳定性        |
| 位置编码                   | 无缩放处理             | 用 learnable 的 2D 位置编码，形状为 `[num_patches, d_model]`<br>直接与 Patch 嵌入相加，同时还带有缩放初始化|
| 输出特征                  | 全序列输入到分类头  | 仅取类别嵌入作为全局特征、传输给输出层      |
| 多模态任务支持             | 无                     | 具备投影层、通过对比学习统一表示空间       |

基于这些关键的总结信息，我们可以开始梳理CLIP中的ViT的全套代码了 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/01.png)

In [53]:
class LayerNorm(nn.LayerNorm):
    """
    LayerNorm 的子类，用于处理 fp16（半精度浮点数）。
    """

    def forward(self, x: torch.Tensor):
        # 保存原始数据类型
        orig_type = x.dtype
        # 将输入转换为 float32 类型，执行 LayerNorm 操作
        ret = super().forward(x.type(torch.float32))
        # 恢复原始数据类型
        return ret.type(orig_type)


class QuickGELU(nn.Module):
    """
    实现 QuickGELU 激活函数，比标准 GELU 更简单且计算效率更高。
    """

    def forward(self, x: torch.Tensor):
        # QuickGELU 激活函数公式：x * sigmoid(1.702 * x)
        return x * torch.sigmoid(1.702 * x)


class ResidualAttentionBlock(nn.Module):
    """
    实现一个残差注意力模块，包括多头注意力机制和前馈网络（MLP）。
    """

    def __init__(self, d_model: int, n_head: int, attn_mask: torch.Tensor = None):
        """
        初始化函数。
        参数：
        - d_model: 模型的隐藏层维度（即特征向量的维度）。
        - n_head: 多头注意力的头数。
        - attn_mask: 注意力掩码，可选。
        """
        super().__init__()

        # 多头注意力模块
        self.attn = nn.MultiheadAttention(d_model, n_head)
        # 第一个 LayerNorm
        self.ln_1 = LayerNorm(d_model)
        # 前馈网络（MLP），包括线性层、QuickGELU 激活和另一层线性变换
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc", nn.Linear(d_model, d_model * 4)),  # 扩展特征维度 4 倍
            ("gelu", QuickGELU()),  # QuickGELU 激活
            ("c_proj", nn.Linear(d_model * 4, d_model))  # 恢复到原始特征维度
        ]))
        # 第二![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/01.png)个 LayerNorm
        self.ln_2 = LayerNorm(d_model)
        # 注意力掩码
        self.attn_mask = attn_mask

    def attention(self, x: torch.Tensor):
        """
        计算多头注意力。
        """
        # 如果存在注意力掩码，将其转换为与输入相同的数据类型和设备
        self.attn_mask = self.attn_mask.to(dtype=x.dtype, device=x.device) if self.attn_mask is not None else None
        # 调用多头注意力，返回注意力结果（不需要权重）
        return self.attn(x, x, x, need_weights=False, attn_mask=self.attn_mask)[0]

    def forward(self, x: torch.Tensor):
        """
        前向传播。
        """
        # 1. 多头注意力：先进行 LayerNorm，再计算注意力，并与输入 x 相加（残差连接）
        x = x + self.attention(self.ln_1(x))
        # 2. 前馈网络：先进行 LayerNorm，再通过 MLP，并与输入 x 相加（残差连接）
        x = x + self.mlp(self.ln_2(x))
        return x


class Transformer(nn.Module):
    """
    基于 ResidualAttentionBlock 的 Transformer 模块。
    """

    def __init__(self, width: int, layers: int, heads: int, attn_mask: torch.Tensor = None):
        """
        初始化函数。
        参数：
        - width: 模型的隐藏层维度。
        - layers: Transformer 层数。
        - heads: 多头注意力的头数。
        - attn_mask: 注意力掩码。
        """
        super().__init__()
        self.width = width
        self.layers = layers
        # 堆叠多个 ResidualAttentionBlock，构成完整的 Transformer
        self.resblocks = nn.Sequential(*[ResidualAttentionBlock(width, heads, attn_mask) for _ in range(layers)])

    def forward(self, x: torch.Tensor):
        """
        前向传播。
        """
        return self.resblocks(x)


class VisionTransformer(nn.Module):
    """
    实现 Vision Transformer，用于处理图像输入。
    """

    def __init__(self, input_resolution: int, patch_size: int, width: int, layers: int, heads: int, output_dim: int):
        """
        初始化函数。
        参数：
        - input_resolution: 输入图像的分辨率（如 224）。
        - patch_size: 图像切片的尺寸（如 16x16）。
        - width: Transformer 的隐藏层维度。
        - layers: Transformer 层数。
        - heads: 多头注意力的头数。
        - output_dim: 输出特征维度。
        """
        super().__init__()
        self.input_resolution = input_resolution
        self.output_dim = output_dim
        # 图像切片卷积，将输入图像分割为若干 patch，并映射到 width 维度
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=width, kernel_size=patch_size, stride=patch_size, bias=False)

        # 类别嵌入，用于标识全局图像特征
        scale = width ** -0.5
        self.class_embedding = nn.Parameter(scale * torch.randn(width))
        # 位置嵌入，用于编码每个 patch 的位置
        self.positional_embedding = nn.Parameter(scale * torch.randn((input_resolution // patch_size) ** 2 + 1, width))
        # 输入的 LayerNorm
        self.ln_pre = LayerNorm(width)

        # Transformer 编码器
        self.transformer = Transformer(width, layers, heads)

        # 输出的 LayerNorm 和投影层
        self.ln_post = LayerNorm(width)
        self.proj = nn.Parameter(scale * torch.randn(width, output_dim))

    def forward(self, x: torch.Tensor):
        """
        前向传播。
        参数：
        - x: 输入图像张量，形状为 (batch_size, 3, H, W)。
        返回值：
        - x: 输出特征，形状为 (batch_size, output_dim)。
        """
        # 图像切片，将输入分割为 patch 并映射到特征维度
        x = self.conv1(x)  # 形状 = [batch_size, width, grid, grid]
        # 将每个 patch 展平，并调整为 [batch_size, num_patches, width]
        x = x.reshape(x.shape[0], x.shape[1], -1)  # 形状 = [batch_size, width, grid ** 2]
        x = x.permute(0, 2, 1)  # 调整为 [batch_size, num_patches, width]

        # 添加类别嵌入
        x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # 形状 = [batch_size, num_patches + 1, width]
        # 加上位置嵌入
        x = x + self.positional_embedding.to(x.dtype)
        # 输入的 LayerNorm
        x = self.ln_pre(x)

        # Transformer 编码器
        x = x.permute(1, 0, 2)  # 调整为 [num_patches + 1, batch_size, width]
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # 调整回 [batch_size, num_patches + 1, width]

        # 输出特征仅取第一个（全局图像特征）
        x = self.ln_post(x[:, 0, :])

        # 如果存在投影层，将其映射到输出维度
        if self.proj is not None:
            x = x @ self.proj

        return x

In [54]:
# 调整参数以减少内存占用
input_resolution = 224  # 分辨率
width = 512  # 隐藏层维度
layers = 6  #  Transformer 层数
heads = 8  # 注意力头数
output_dim = 512  # 输出特征维度
batch_size = 10  # batch大小

# 重新定义 VisionTransformer 模型
vit = VisionTransformer(
    input_resolution=input_resolution,
    patch_size=16,
    width=width,
    layers=layers,
    heads=heads,
    output_dim=output_dim
)

In [55]:
# 假设输入图像数据 (10,3,224,224)
input_data = torch.randn(batch_size, 3, input_resolution, input_resolution)

# 通过模型前向传播
output = vit(input_data)

# 打印输出的形状
output.shape

torch.Size([10, 512])

如你缩减，我们将结构为(10,3,224,224)的图像数据压缩为了二维数据(10,128)，这和ResNet的输出结果一致。

### 1.2 文本编码器速览

在 **CLIP** 中，Transformer 作为文本编码器，其结构与标准的 Transformer 中的编码器结构高度相似，由嵌入层（Embedding Layer）、多层的自注意力模块（Self-Attention）、前馈网络（Feed-Forward Network, FFN）和归一化（LayerNorm）组成。

首先，输入文本通过嵌入层，将每个词映射为固定维度的向量，同时添加可学习的 **位置编码**，以保留序列的位置信息。随后，这些嵌入通过多层 Transformer 模块，每层由多头自注意力（Multi-Head Attention）和 FFN 组成，并在每个子模块后加上残差连接（Residual Connection）和 LayerNorm。Transformer 的作用是逐层捕获文本序列中的上下文关系和全局语义特征。最终，文本的全局语义特征通过特定的标记（例如 [EOS] token）的表示提取，并投影到与图像特征一致的嵌入空间中，用于多模态匹配。这种结构使 CLIP 能够高效地对齐图像和文本的语义信息，实现跨模态理解。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/01.png)

## 2 CLIP模型的预训练流程

现在我们有了图像信息和文本信息，但一个多模态模型如何将这两类信息整合起来呢？就像文字架构预训练目标是创造出“能够理解基础文字信息”、多模态架构的预训练目标就是“能够理解基础文字和基础图像信息”。因此，CLIP的预训练流程是让架构本身能够理解图像和文本的流程。**CLIP 通过学习图像和文字的匹配关系，成功地将两种模态的信息（图像和文本）映射到同一个语义空间，这使得它能够理解并关联图像和文本**，从而实现强大的多模态任务能力，比如跨模态检索、分类、生成等。

在 CLIP 中，计算余弦相似度和匹配图像与文本特征的过程可以概括为：
1. 输入**图-文对数据集**作为初始数据、各自提取图像和文本特征。<br><br>
2. 通过归一化、点积等流程，计算图像和文本的**配对余弦相似度**。<br><br>
3. 实现**对比学习**（Contrastive Learning）、使用对比损失作为损失函数，对比损失越高说明匹配越不精准——
> 基于损失函数来调整**相似度矩阵**，调整的方向是让配对样本的余弦相似度尽可能接近 1，而非配对样本尽可能接近 0。<br><br>
> 由于要推动相似度矩阵变得更精准、因此损失函数也会**优化图像编码结果和文字编码结果**（也就是优化图像编码器和文字编码器的权重），如果样本是配对的、那拉进两种编码结果在语义空间中更接近，如果样本不是配对的、则将样本的编码结果在语义空间中分离。<br>

4. 最终，将图像和文本的语义表示统一到一个多模态的共享空间中、使得图像和文字能够被精准理解，例如通过输入文本检索相关图像，或者通过图像查找相关文本描述，实现高效的跨模态搜索和匹配。

---

### 2.1 学习预训练流程的意义

1. **为了与分布式框架进行联用**：当使用 DeepSpeed 或 Accelerate 来实现类似 CLIP 的算法训练时、框架主要负责优化分布式训练流程，实际的训练代码是需要我们提供的，包括——
- 如何定义模型
- 如何分发数据
- 如何处理损失计算和梯度更新

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/07.png)

2. **每个算法的预训练流程都不同**：包括但不限于独特的数据处理流程、独特的模型设计、独特的架构选择、独特的损失函数设计、独特的资源优化方法等等，为了能够训练各种各样的模型，我们需要对模型的预训练方式进行特定的研究。

3. **为了更高效地调整训练超参数**：每个模型的预训练流程都对特定的超参数（如学习率、批次大小、温度参数等）有不同的敏感性。如果不了解模型的预训练细节，就很难选择合适的超参数。

4. **为了更好地迁移到下游任务**：预训练模型的效果直接影响迁移到下游任务时的表现。如果对预训练流程不了解，可能会误用模型的输出或特征（如未正确归一化嵌入向量）。

### 2.2 经典图文对数据集

在多模态研究中，有许多经典的图文对数据集可用于预训练，涵盖英文和中文的多模态场景——

**英文图文对数据集**
1. **COCO Captions**  
   - **简介**：MS COCO 数据集的扩展版本，包含图像和其多种文本描述。每张图像对应 5 条人工撰写的文本。
   - **规模**：约 330k 图像-文本对。
   - **应用**：图像描述生成（Captioning）、多模态检索、图文匹配。
   - **链接**：[COCO Captions](https://cocodataset.org/#captions-2015)
<br><br>
2. **Flickr30k**  
   - **简介**：包含日常场景的图片及其文本描述，每张图像有 5 条人工标注。
   - **规模**：31k 图像-文本对。
   - **应用**：图文匹配、跨模态检索。
   - **链接**：[Flickr30k](http://shannon.cs.illinois.edu/DenotationGraph/)
<br><br>
3. **Visual Genome**  
   - **简介**：包含详细的图像标注和关系描述，每张图像对应多个区域和其文字说明。
   - **规模**：约 108k 图像，超过 500 万区域描述。
   - **应用**：图像场景理解、关系推理、多模态检索。
   - **链接**：[Visual Genome](https://visualgenome.org/)
<br><br>
4. **Conceptual Captions (CC3M & CC12M)**  
   - **简介**：从网页爬取的大规模图文对数据集，使用自动化方法生成文本描述。
   - **规模**：CC3M 包含 3M 图文对，CC12M 包含 12M 图文对。
   - **应用**：多模态预训练、图文生成、检索。
   - **链接**：[Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/)
<br><br>
5. **LAION-400M / LAION-5B**  
   - **简介**：一个超大规模的开放图文对数据集，包含从互联网中爬取的图文对，经过一定程度的清理。
   - **规模**：LAION-400M（4 亿对），LAION-5B（50 亿对）。
   - **应用**：多模态预训练、检索、生成任务。
   - **链接**：[LAION](https://laion.ai/)

---

**中文图文对数据集**

1. **WenLan (文澜)**  
   - **简介**：中文多模态预训练数据集，由多个来源的大规模图文对组成，涵盖自然图片、新闻图片等场景。
   - **规模**：约 30M 图文对。
   - **应用**：多模态预训练、中文图文检索。
   - **论文**：[WenLan](https://arxiv.org/abs/2103.06561)
<br><br>
2. **CLUEVIT**  
   - **简介**：中文领域的 Vision Transformer 数据集，适用于多模态任务。
   - **规模**：超过 10M 图文对。
   - **应用**：多模态检索、图文生成。
   - **链接**：[CLUE](https://www.cluebenchmarks.com/)
<br><br>
3. **CCF Open Dataset**  
   - **简介**：由中文社区提供的多模态开放数据集，部分数据集包含图文对。
   - **规模**：根据具体子集而定。
   - **应用**：多模态任务研究。
   - **链接**：[CCF](https://www.ccf.org.cn/)
<br><br>
4. **AiM Dataset**  
   - **简介**：一个中文图文对数据集，包含社交媒体图片及其描述。
   - **规模**：待公开具体规模。
   - **应用**：多模态生成、检索。

---

**如何选择合适的数据集**

1. **任务需求**：<br><br>
   - 图文匹配或检索：COCO Captions、Flickr30k、WenLan。
   - 多模态生成：CC12M、Visual Genome。
   - 多模态预训练：LAION、Conceptual Captions、WenLan。
<br><br>
2. **语言和领域**：<br><br>
   - 英文场景：使用 LAION 或 Conceptual Captions。
   - 中文场景：推荐使用 WenLan 或 CLUEVIT。
<br><br>
3. **数据规模**：<br><br>
   - 小规模：COCO Captions（约 330k 对）。
   - 大规模：LAION-5B、WenLan（适合预训练大模型）。

### 2.3 余弦相似度

在 **CLIP（Contrastive Language–Image Pretraining）** 模型中，图像和文本特征的匹配是核心任务，而 **余弦相似度** 是实现这一匹配的关键工具。CLIP 的目标是通过对比学习，将配对的图像和文本映射到同一语义空间，使其具有相似的特征表示。在CLIP中，我们计算余弦相似度是为了**衡量图像和文本的相似性**。经过图像的编码器和文本的编码器后，图像和文本的特征表示都是高维向量，我们计算它们之间的余弦相似度衡量它们之间的语义相关性。通过余弦相似度，CLIP 区分语义匹配的图像-文本对和不匹配的对。

- **余弦相似度的计算**

余弦相似度（Cosine Similarity）衡量两个向量之间的相似度，通过计算它们夹角的余弦值，具体公式为：

$$
\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$

其中：
- $ A \cdot B $ 是向量 $ A $ 和 $ B $ 的点积。
- $ \|A\| $ 是向量 $ A $ 的范数（即向量的长度，定义为 $ \sqrt{\sum_i A_i^2} $）。
- $ \|B\| $ 是向量 $ B $ 的范数。

这里的结果直接反映了两个向量之间的方向关系：
- 当 $ \cos\theta = 1 $ 时，向量同向。
- 当 $ \cos\theta = 0 $ 时，向量正交。
- 当 $ \cos\theta = -1 $ 时，向量反向。

当考虑语义和图像的信息时，我们关注的是包含的信息是否同义（向量的方向是否相同），而非信息总量是否一致（向量的大小是否一致）。通常来说，图像向量所包含的信息是要高于文字向量的。

In [1]:
import numpy as np

def cosine_similarity(A, B):
    """
    计算两个向量A和B之间的余弦相似度。

    参数：
    A, B -- 数组形式的向量

    返回值：
    cos_sim -- 余弦相似度值
    """
    dot_product = np.dot(A, B)          # 计算点积
    norm_A = np.linalg.norm(A)          # 计算A的范数
    norm_B = np.linalg.norm(B)          # 计算B的范数
    cos_sim = dot_product / (norm_A * norm_B)
    return cos_sim

In [2]:
# 示例向量
A = np.array([1, 2, 3])
B = np.array([1, 2, 2.9])   # 与A方向相近
C = np.array([3, -1, 0])    # 与A方向较远

# 计算余弦相似度
cos_sim_AB = cosine_similarity(A, B)
cos_sim_AC = cosine_similarity(A, C)

print(f"A和B的余弦相似度: {cos_sim_AB}")
print(f"A和C的余弦相似度: {cos_sim_AC}")

A和B的余弦相似度: 0.9998668282478015
A和C的余弦相似度: 0.08451542547285165


从结果来看，$ A $ 与 $ B $ 的余弦相似度：**0.9977**，非常接近 1，说明 $ A $ 和 $ B $ 的方向几乎一致。而$ A $ 与 $ C $ 的余弦相似度：**0.0845**，接近 0，说明 $ A $ 和 $ C $ 的方向几乎正交（不相关）。

- CLIP中的余弦相似度

CLIP的输入数据是**一一配对的图文对**，例如，图像 $ I_i $ 和对应的描述 $ T_i $。这些图文对通常是从真实数据集中构建的，反映了某种语义上的匹配关系，比如图像是一只狗，而文本是“a photo of a dog”。在将数据递给CLIP用于训练时，我们会给成千上万的图文对、而不是一对图文，**因此我们要计算的是余弦相似度矩阵、而不是单一的余弦相似度**。

假设一个训练批次有 $ N = 3 $ 个图文对：
- 图像：$ I_1, I_2, I_3 $
- 文本：$ T_1, T_2, T_3 $

这3个图文对输入CLIP后、会分别得到3个图像向量、3个文本向量，基于这6个向量，我们需要计算**任意图向量和文向量之间的余弦相似度**。也就是说，我们要求解的是——
$$ S_{i,j} = \text{cosine\_similarity}(I_i, T_j) $$

求解出的是一个$ N \times N $ 的相似度矩阵 $ S $——

$$
    S =
    \begin{bmatrix}
    S_{1,1} & S_{1,2} & \dots & S_{1,N} \\
    S_{2,1} & S_{2,2} & \dots & S_{2,N} \\
    \vdots  & \vdots  & \ddots & \vdots \\
    S_{N,1} & S_{N,2} & \dots & S_{N,N}
    \end{bmatrix}
$$

在 $ N = 3 $ 的情况下，我们的矩阵具体是——

$$
S =
\begin{bmatrix}
S_{1,1} & S_{1,2} & S_{1,3} \\
S_{2,1} & S_{2,2} & S_{2,3} \\
S_{3,1} & S_{3,2} & S_{3,3}
\end{bmatrix}
$$

其中——
- $ S_{i,j} $ 表示第 $ i $ 个图像 $ I_i $ 和第 $ j $ 个文本 $ T_j $ 的余弦相似度。
- 对角线元素（如 $ S_{1,1}, S_{2,2}, S_{3,3} $）是配对的**正样本**，CLIP预训练的优化目标是让它们变大。
- 非对角线元素（如 $ S_{1,2}, S_{1,3}, S_{2,1} $ 等）是非配对的**负样本**，CLIP预训练的优化目标是让它们变小。

In [3]:
import numpy as np

# 假设图像编码器和文本编码器的输出矩阵 (3, 512)
np.random.seed(42)  # 固定随机种子以便复现
image_features = np.random.rand(3, 512)  # 模拟图像编码器输出
text_features = np.random.rand(3, 512)  # 模拟文本编码器输出

# 在进行余弦相似度计算之前、一般会对图像和文本特征都进行L2归一化
def l2_normalize(matrix):
    return matrix / np.linalg.norm(matrix, axis=1, keepdims=True)

image_features_normalized = l2_normalize(image_features)
text_features_normalized = l2_normalize(text_features)

# 计算余弦相似度矩阵 S_ij
similarity_matrix = np.dot(image_features_normalized, text_features_normalized.T)

# 展示结果
similarity_matrix

array([[0.73934186, 0.76573416, 0.76050724],
       [0.75515615, 0.73161239, 0.73684385],
       [0.75104627, 0.76569744, 0.73533973]])

### 2.4 对比学习 Contrastive Learning

- **infoNCE损失**

对比学习是一种基于表示学习（Representation Learning）的**预训练技术**，通过构造正样本和负样本的对比任务，学习将相似的数据拉近、不相似的数据推远的特征表示。它被广泛应用于无监督学习、半监督学习以及多模态学习（如CLIP）中。在深度学习的任何预训练方法中，包括对比学习、生成式学习或监督学习，**核心流程都是找到一个适合的损失函数，然后通过最大化或最小化损失函数来调整模型权重**。因此我们在任何预训练中都需要高度关注损失函数。

在CLIP使用对比学习时，我们架构的最终输出会被计算余弦相似度，而我们常用的损失是对比损失（例如，InfoNCE损失或Triplet损失）。典型的对比损失函数是**InfoNCE损失**（Information Noise-Contrastive Estimation，信息噪音对比估计损失），其公式如下：

$$
\mathcal{L}_{i} = -\log \frac{\exp(\text{sim}(z_i, z_i^+)/\tau)}{\sum_{j=1}^N \exp(\text{sim}(z_i, z_j)/\tau)}
$$

其中：
- $ z $ 是任意模态信息的某种嵌入表示，也就是我们之前提到的、CLIP输出的$T_i$或者$I_i$。
<br><br>
- $ \text{sim}(\cdot, \cdot) $ 是相似度（如余弦相似度），在当前公式上，分子上的$\text{sim}(z_i, z_i^+)$表示匹配的图文对计算出的余弦相似度（正样本计算出的余弦相似度），而分母上的$\text{sim}(z_i, z_j)$则表示不匹配的图文对计算出的余弦相似度（也就是负样本计算出的余弦相似度）。
<br><br>
- $ \tau $ 是温度超参数，用于调节分布的平滑性。

因此我们也可以将这个公式简化为——

$$
\mathcal{L}_{i} = -\log \frac{\exp(S_{ii}/\tau)}{\sum_{j=1}^N \exp(S_{ij}/\tau)}
$$

要如何理解这个公式？如你所见，公式的$L_i$代表了一张图或者一段文字的InfoNCE，其中分子上是当前图像/文字与配对的文/图向量之间的余弦相似度，分母上是当前图像/文字与不配对的所有文字/图像向量所构成的余弦相似度之和。因此，对当前图/文向量来说，分子上是该图文对应的唯一的正样本的余弦相似度、分母上是其他所有的负样本的余弦相似度之和。整体来看，$L_i$**表示当前样本与正样本的相似度在所有候选样本相似度分布中的比例**。这个比例越大，表示模型越具备准确的理解力和判断力。

在这个公式中，我们希望的损失函数迭代方向是**分子大、分母小**，具体地来说——

- **分子大**：
  - 希望正样本的相似度 $ S_{ii} $ 尽可能高（分子越大），这会鼓励模型学习正样本对的嵌入表示更接近。

- **分母小（相对而言）**：
  - 希望负样本的相似度 $ S_{ij} $（$ j \neq i $）尽可能低（分母相对减小），这会迫使模型将负样本与当前样本拉远。

优化的核心是让正样本对在分布中占据尽可能大的权重，从而最大化分子的贡献，最小化分母中负样本的干扰。**因此我们是希望最大化该比例的、即最大化正样本的相对权重**。在此基础上，我们在该比例的外层放上 $ -\log $ 转换，这是对数概率的一种变换，这可以帮助我们将优化目标由“最大化相对比例”转换为最小化损失值（即典型的深度学习目标）。

$$
-\log p_i = -\log \frac{\exp(S_{ii}/\tau)}{\sum_{j=1}^N \exp(S_{ij}/\tau)}
$$

这种转化方式类似于在需要最大化的公式前加上负号、实现负对数似然。同时，如果 $ p_i $ （正样本的相对权重）很小，$ -\log(p_i) $ 会快速变大，从而对模型优化造成更大的惩罚。这意味着模型会更加努力去提高 $ p_i $、更进一步接近我们的目标。除此之外，负对数变换还能将乘法形式的优化目标转换为加法形式（对概率分布进行对数变换），简化了梯度计算。

- CLIP所使用的对比损失Contrastive Loss

现在你已经了解了infoNCE，相信你一定有注意到它是某个特定的样本（图向量或文向量）下的损失。在CLIP中，我们使用infoNCE拓展出了 **对比损失（Contrastive Loss）**，这个对比损失包括以下两部分：

1. **图像到文本的匹配**：
   - 对每个图像的特征 $ I_i $ 与所有文本特征 $ T_j $ 求解余弦相似度，每个图像都会有自己的正样本占比，对所有这些正样本占比求均值——
     $$
     \mathcal{L}_{\text{image}} = - \frac{1}{N} \sum_{i=1}^{N} \log \frac{\exp(S_{ii} / \tau)}{\sum_{j=1}^{N} \exp(S_{ij} / \tau)}
     $$

2. **文本到图像的匹配**：
   - 类似地，将每个文本的特征 $ T_j $ 与所有图像特征 $ I_i $ 求解余弦相似度、每个文本都会有自己的正样本占比，对所有这些正样本占比求均值——
     $$
     \mathcal{L}_{\text{text}} = - \frac{1}{N} \sum_{j=1}^{N} \log \frac{\exp(S_{jj} / \tau)}{\sum_{i=1}^{N} \exp(S_{ij} / \tau)}
     $$

3. **总损失**：
   - 最终对比损失是两者的平均：
     $$
     \mathcal{L} = \frac{1}{2} (\mathcal{L}_{\text{image}} + \mathcal{L}_{\text{text}})
     $$
   - 其中，$\tau$ 是一个可学习的缩放参数（在代码中用 `logit_scale` 表示），控制相似度的数值范围。而整个CLIP的对比学习预训练过程中的目标就是最小化$L$。

**完整的CLIP实现代码如下——**

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import Tuple, Union

class CLIP(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 # vision
                 image_resolution: int,
                 vision_layers: Union[Tuple[int, int, int, int], int],
                 vision_width: int,
                 vision_patch_size: int,
                 # text
                 context_length: int,
                 vocab_size: int,
                 transformer_width: int,
                 transformer_heads: int,
                 transformer_layers: int
                 ):
        super().__init__()

        self.context_length = context_length

        if isinstance(vision_layers, (tuple, list)):
            vision_heads = vision_width * 32 // 64
            self.visual = ModifiedResNet(
                layers=vision_layers,
                output_dim=embed_dim,
                heads=vision_heads,
                input_resolution=image_resolution,
                width=vision_width
            )
        else:
            vision_heads = vision_width // 64
            self.visual = VisionTransformer(
                input_resolution=image_resolution,
                patch_size=vision_patch_size,
                width=vision_width,
                layers=vision_layers,
                heads=vision_heads,
                output_dim=embed_dim
            )

        self.transformer = Transformer(
            width=transformer_width,
            layers=transformer_layers,
            heads=transformer_heads,
            attn_mask=self.build_attention_mask()
        )

        self.vocab_size = vocab_size
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)
        self.positional_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))
        self.ln_final = LayerNorm(transformer_width)

        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

        self.initialize_parameters()

    def initialize_parameters(self):
        nn.init.normal_(self.token_embedding.weight, std=0.02)
        nn.init.normal_(self.positional_embedding, std=0.01)

        if isinstance(self.visual, ModifiedResNet):
            if self.visual.attnpool is not None:
                std = self.visual.attnpool.c_proj.in_features ** -0.5
                nn.init.normal_(self.visual.attnpool.q_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.k_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.v_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.c_proj.weight, std=std)

            for resnet_block in [self.visual.layer1, self.visual.layer2, self.visual.layer3, self.visual.layer4]:
                for name, param in resnet_block.named_parameters():
                    if name.endswith("bn3.weight"):
                        nn.init.zeros_(param)

        proj_std = (self.transformer.width ** -0.5) * ((2 * self.transformer.layers) ** -0.5)
        attn_std = self.transformer.width ** -0.5
        fc_std = (2 * self.transformer.width) ** -0.5
        for block in self.transformer.resblocks:
            nn.init.normal_(block.attn.in_proj_weight, std=attn_std)
            nn.init.normal_(block.attn.out_proj.weight, std=proj_std)
            nn.init.normal_(block.mlp.c_fc.weight, std=fc_std)
            nn.init.normal_(block.mlp.c_proj.weight, std=proj_std)

        if self.text_projection is not None:
            nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)

    def build_attention_mask(self):
        # lazily create causal attention mask, with full attention between the vision tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = torch.empty(self.context_length, self.context_length)
        mask.fill_(float("-inf"))
        mask.triu_(1)  # zero out the lower diagonal
        return mask

    @property
    def dtype(self):
        return self.visual.conv1.weight.dtype

    def encode_image(self, image):
        return self.visual(image.type(self.dtype))

    def encode_text(self, text):
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection

        return x

    def forward(self, image, text):
        image_features = self.encode_image(image)
        text_features = self.encode_text(text)

        # normalized features
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()

        # shape = [global_batch_size, global_batch_size]
        return logits_per_image, logits_per_text


def convert_weights(model: nn.Module):
    """Convert applicable model parameters to fp16"""

    def _convert_weights_to_fp16(l):
        if isinstance(l, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            l.weight.data = l.weight.data.half()
            if l.bias is not None:
                l.bias.data = l.bias.data.half()

        if isinstance(l, nn.MultiheadAttention):
            for attr in [*[f"{s}_proj_weight" for s in ["in", "q", "k", "v"]], "in_proj_bias", "bias_k", "bias_v"]:
                tensor = getattr(l, attr)
                if tensor is not None:
                    tensor.data = tensor.data.half()

        for name in ["text_projection", "proj"]:
            if hasattr(l, name):
                attr = getattr(l, name)
                if attr is not None:
                    attr.data = attr.data.half()

    model.apply(_convert_weights_to_fp16)


def build_model(state_dict: dict):
    vit = "visual.proj" in state_dict

    if vit:
        vision_width = state_dict["visual.conv1.weight"].shape[0]
        vision_layers = len([k for k in state_dict.keys() if k.startswith("visual.") and k.endswith(".attn.in_proj_weight")])
        vision_patch_size = state_dict["visual.conv1.weight"].shape[-1]
        grid_size = round((state_dict["visual.positional_embedding"].shape[0] - 1) ** 0.5)
        image_resolution = vision_patch_size * grid_size
    else:
        counts: list = [len(set(k.split(".")[2] for k in state_dict if k.startswith(f"visual.layer{b}"))) for b in [1, 2, 3, 4]]
        vision_layers = tuple(counts)
        vision_width = state_dict["visual.layer1.0.conv1.weight"].shape[0]
        output_width = round((state_dict["visual.attnpool.positional_embedding"].shape[0] - 1) ** 0.5)
        vision_patch_size = None
        assert output_width ** 2 + 1 == state_dict["visual.attnpool.positional_embedding"].shape[0]
        image_resolution = output_width * 32

    embed_dim = state_dict["text_projection"].shape[1]
    context_length = state_dict["positional_embedding"].shape[0]
    vocab_size = state_dict["token_embedding.weight"].shape[0]
    transformer_width = state_dict["ln_final.weight"].shape[0]
    transformer_heads = transformer_width // 64
    transformer_layers = len(set(k.split(".")[2] for k in state_dict if k.startswith("transformer.resblocks")))

    model = CLIP(
        embed_dim,
        image_resolution, vision_layers, vision_width, vision_patch_size,
        context_length, vocab_size, transformer_width, transformer_heads, transformer_layers
    )

    for key in ["input_resolution", "context_length", "vocab_size"]:
        if key in state_dict:
            del state_dict[key]

    convert_weights(model)
    model.load_state_dict(state_dict)
    return model.eval()

一套逻辑完整、但实现简单的训练代码——

```python

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from tqdm import tqdm

# 定义 CLIP 模型（假设已提供 CLIP 的实现代码）
# from your_clip_implementation import CLIP, build_model, clip_loss

# 定义数据集类
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, captions, transform):
        self.image_paths = image_paths
        self.captions = captions
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)  # 返回数据集的大小

    def __getitem__(self, idx):
        image = self.transform(self.image_paths[idx])  # 对图像进行变换
        text = self.captions[idx]  # 获取对应的文本（已进行分词处理）
        return image, text

# 定义 CLIP 的损失函数
def clip_loss(image_features, text_features, temperature):
    image_features = F.normalize(image_features, dim=-1)  # 对图像特征进行 L2 归一化
    text_features = F.normalize(text_features, dim=-1)    # 对文本特征进行 L2 归一化

    logits = torch.matmul(image_features, text_features.T) / temperature  # 计算相似度矩阵并缩放
    labels = torch.arange(logits.size(0)).to(logits.device)  # 正样本标签为对角线上的元素

    loss_image = F.cross_entropy(logits, labels)  # 图像到文本的交叉熵损失
    loss_text = F.cross_entropy(logits.T, labels)  # 文本到图像的交叉熵损失

    return (loss_image + loss_text) / 2  # 返回平均损失

# 定义训练函数
def train_clip(
    model, dataloader, optimizer, scheduler, device, num_epochs=10, temperature=0.07
):
    model.train()  # 设置模型为训练模式
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):  # 使用 tqdm 显示训练进度
            images, texts = batch
            images, texts = images.to(device), texts.to(device)  # 将数据加载到设备（GPU 或 CPU）

            optimizer.zero_grad()  # 梯度清零
            image_features = model.encode_image(images)  # 提取图像特征
            text_features = model.encode_text(texts)    # 提取文本特征

            loss = clip_loss(image_features, text_features, temperature)  # 计算损失
            loss.backward()  # 反向传播

            optimizer.step()  # 更新参数
            if scheduler:  # 如果有学习率调度器，更新学习率
                scheduler.step()

            epoch_loss += loss.item()  # 累计当前批次的损失

        print(f"Epoch {epoch + 1}: Loss = {epoch_loss / len(dataloader):.4f}")  # 输出当前轮次的平均损失

# 定义数据增强和预处理
transform = Compose([
    Resize((224, 224)),  # 调整图像大小
    ToTensor(),          # 将图像转换为张量
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 归一化
])

# 示例数据集（图像使用 ImageFolder，文本使用对应的分词数据）
# 替换为实际数据集路径和分词后的文本数据
image_dataset = ImageFolder(root="path_to_images", transform=transform)
text_data = ["a caption for image 1", "a caption for image 2", "a caption for image 3"]  # 替换为实际文本数据
dataset = CLIPDataset(image_dataset.imgs, text_data, transform)

# 数据加载器
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 检测是否有可用的 GPU
model = build_model(state_dict=torch.load("path_to_pretrained_model.pth"))  # 替换为实际的模型路径
model = model.to(device)  # 将模型加载到设备

# 优化器和学习率调度器
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)  # AdamW 优化器
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # 学习率调度器

# 开始训练模型
train_clip(model, dataloader, optimizer, scheduler, device, num_epochs=10, temperature=0.07)
```

## 3 Llava模型架构

**LLaVA（Large Language and Vision Assistant）** 是一种结合预训练视觉编码器和生成式语言模型的多模态架构，旨在实现视觉与语言的高效对齐与交互。LLaVA 的核心由三个主要模块组成：首先，通过 CLIP 的 ViT-L/14 视觉编码器提取图像特征，这些特征通过一个可训练的线性投影矩阵 $ W $ 映射到语言模型的嵌入空间，从而生成视觉嵌入 $ H_v $。其次，生成式语言模型（如 Vicuna）接收视觉嵌入 $ H_v $ 和语言指令 $ X_{\text{instruct}} $ 作为输入，以自回归方式生成多模态任务的自然语言响应 $ X_a $。这种设计充分利用了预训练模型的能力，并通过轻量级投影模块实现视觉特征与语言模型的无缝对接。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/09.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/08.png)

**LLaVA（Large Language and Vision Assistant）** 的架构设计十分简单明了，但它的核心思想非常巧妙。整体 LLaVA 的架构可以概括为以下几个部分：

1. **视觉编码器（Vision Encoder）**：<br><br>
   - 使用了预训练的 **CLIP 视觉编码器（ViT-L/14）** 来提取图像的视觉特征。
   - 图像 $ X_v $ 被输入到 CLIP 视觉编码器中，提取的视觉特征记为 $ Z_v $：
     $$
     Z_v = g(X_v)
     $$
   - 提取的特征可以是 **最后一个 Transformer 层的特征** 或 **中间层的网格特征（grid features）**。
<br><br>
2. **投影层（Projection Layer）**：<br><br>
   - 通过一个可训练的线性投影矩阵 $ W $，将视觉特征 $ Z_v $ 转换为与语言模型嵌入相同维度的嵌入 $ H_v $：
     $$
     H_v = W \cdot Z_v
     $$
   - **目的**：使视觉特征能够与语言模型的词嵌入在相同的语义空间中对齐，从而使语言模型可以理解视觉特征。
<br><br>
3. **语言模型（Language Model, LLM）**：<br><br>
   - 使用了 **预训练的大语言模型（如 Vicuna）**，记为 $ f_\phi $。
   - **输入**：
     - 转换后的视觉嵌入 $ H_v $。
     - 语言指令 $ X_q $。
   - **输出**：语言模型生成的语言响应 $ X_a $，完成基于视觉和语言的多模态任务。
<br><br>
4. **多模态交互**：<br><br>
   - 将视觉嵌入 $ H_v $ 和语言指令嵌入 $ H_q $ 按顺序组合，作为语言模型的输入，利用语言模型的生成能力完成多模态任务。

LLaVA 采用两阶段的训练流程：第一阶段，通过冻结语言模型和视觉编码器的权重，仅训练投影矩阵，实现视觉与语言空间的对齐；第二阶段，通过端到端微调，进一步优化投影矩阵和语言模型的权重，用于开发多模态聊天机器人和复杂科学问答任务。LLaVA 架构以其简单高效的设计展现出在多模态任务中的强大表现，为实现通用的视觉与语言交互提供了新思路。